# Generating Graph-based heirarchies via eigenvector decomposition

This script is about extracting more data from the fact that the adjacency matrix, containing the eighted edges between nodes, can be diagonalised tos yield a set of eigenvector ($v_\lambda$) and eigvenvalues ($\lambda$). This application is called **spectral graph theory**: https://en.wikipedia.org/wiki/Spectral_graph_theory

The first eigenvalue $\lambda_1$ extracts the eigenvector centrality ($v_{\lambda1}$), which also guarantees that all of $v_{\lambda1}$ elements are >=0. This is convenient for many node-weighted analyses that don't play well with negative weights. Like most authors, we interpret this to be the influence graph - however, it is importance that it is only the influence of the **largest cohesive domain within the protein**. You will often note that for multidomain protein graphs, the centrality often identified only one domain, with all others as almost zero.

This is because spectral decomposition actually partitions the graph into functioning subunits. The negative values of subsequent $v_\lambda$ tells us 
something important about the intra-graph dynamics. Like the harmonics of a vibrating string, discrete domains will tend to anti-correlate so as to preserve the linear independence of spectral decomposition.

Thus, the relative values of domains tells us something important about the cooperativity between domains. Graph-wise, the level of interconnectedness. This scales from zero for completely independent subunits to some as yet unknown maximum for a single domain protein.

*Note: This is the MD-equivalent  of Kundu et al. (2006)'s work on using Gaussian Network Models to decompose proteins into discrete domains.*

*Also c.f. Mishra et al. (2018) as they compare the correlation networked generated from MD to GNM, and find that they largely agree.*

In [ ]:
# Load the python package
import os
from dynetan.toolkit import *
from dynetan.viz import *
from dynetan.proctraj import *
from dynetan.gencor import *# Load the python package
import os
from dynetan.toolkit import *
from dynetan.viz import *
from dynetan.proctraj import *
from dynetan.gencor import *
from dynetan.contact import *
from dynetan.datastorage import *

import numpy as np
import pandas as pd

In [ ]:
# For visualization
from bokeh.io import output_file, output_notebook, push_notebook, show
from bokeh import models as bokehModels
from bokeh import transform as bokehTransform
from bokeh import layouts as bokehLayouts
from bokeh import plotting as bokehPlotting
from bokeh import palettes as bokehPalettes
from bokeh import events as bokehEvents
# For pre-calculating CArtesian distances based on 2D embedding
from sklearn.manifold import MDS

In [ ]:
def convert_booleans_to_binary(arrBools):
    return np.multiply(arrBools,np.power(2,np.arange(len(listBools)-1,-1,-1)))

def extract_spectral_heirarchy(v, nVec=2, bCompress=False):
    """
    Assumes right eigenvectors for now with v[:,i] corresponding to eigenvalue i
    """
    tempArr = np.zeros( (v.shape[0],nVec), dtype='bool' )
    tempArr = v[:,:nVec] < 0
    outArr  = np.zeros( (v.shape[0],nVec), dtype='int' )
    for i in range(1,nVec):
        # = = = Rank the highest eigenvector as the smallest contributor to the binary heirarchical decomposition
        powers = np.power(2,np.arange(i,-1,-1))*2**(nVec-i-1)
        #print( i, powers)
        outArr[:,i] = np.sum( np.multiply( tempArr[:,:i+1], powers ), axis=1)
        
    if bCompress:
        dictMap = { b:a for a,b in enumerate(np.unique(outArr)) }
        for i in range(outArr.shape[0]):
            for j in range(outArr.shape[1]):
                outArr[i,j]=dictMap[outArr[i,j]]
        return outArr
    else:
        return outArr

def compute_eigenvector_overlap_inner(v1,v2):
    return np.sum(np.fabs(np.multiply(v1,v2)))
    #return np.dot(np.square(v1),np.square(v2))
    #return np.dot(np.fabs(v1),np.fabs(v2))
    #return np.dot(v1,v2)

def compute_eigenvector_overlap(v, nVec=2):
    outArr = np.zeros((nVec,nVec))
    for i in range(nVec):
        outArr[i,i]=compute_eigenvector_overlap_inner(v[:,i],v[:,i])
        
    for i in range(nVec-1):
        for j in range(i+1,nVec):
            outArr[i,j]=outArr[j,i]=compute_eigenvector_overlap_inner(v[:,i],v[:,j])
    
    return outArr

In [ ]:
def build_structure_dataframe(atomSel, key, style='VMD'):
    """
    Returns a 3-column pandas data frame containing the segid, resid, and name for use in exporting.
    This makes selecting the atoms in a visualiser much easier
    """
    dataBlock = np.stack( (key, atomSel.segids,atomSel.resids,atomSel.names), axis=1)
    if style == 'VMD':
        colNames = ['key','segname','resid','name']
    elif style == 'MDA':
        colNames = ['key','segid','resid','name']
    return pd.DataFrame( dataBlock, columns = colNames )

## Parameter setting for terminal and notebook formats

In [ ]:
bPythonExport = False

In [ ]:
if bPythonExport:
    import argparse
    parser = argparse.ArgumentParser(description='Perform graph spectral decomposition on a pre-computed DNAD file.',
                                                 formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('-n', '--num_vecs', type=int, dest='numVecs', default=4,
                        help='Number of eigenvectors to report. Noting that the first eigenvector is the canonical centrality, '
                             'with higher components denoting divisions into influential sub-domains.')
    parser.add_argument('-i', '--input', type=str, dest='inputPrefix', default=None, required=True,
                        help='Input prefix used to generate files from Step 1.')
    parser.add_argument('-o', '--output', type=str, dest='outputPrefix', default=None, required=True,
                        help='The prefix for all output files, included HTML visualisations aand data files')
    parser.add_argument('-r','--ref', type=str, dest='referencePrefix', default=None,
                        help='(Not currently used) An equivalent input prefix to include a reference Step 1 computation for comparison.')
    parser.add_argument('--title', type=str, dest='titleGraph', default='Spectral Decomposition',
                        help='Name of the graph titles to be shown in the output HTML.')
    
    args = parser.parse_args()
    
    fullPathRoot   = args.inputPrefix
    filePDBRef = fullPathRoot+"_reducedTraj.pdb"
    outputPrefix = args.outputPrefix
    nVecs = args.numVecs
    titleGraph     = args.titleGraph
    bComparison = False
    if args.referencePrefix is not None:
        fullPathRootB = args.referencePrefix
        filePDBRefB = fullPathRootB+"_reducedTraj.pdb"
        bComparison = True

In [ ]:
# = = = Change to the relevant work folder
if not bPythonExport:
    systemExample="periplasmic"
    if systemExample == "UbqCHARMM" or systemExample == "UbqTempProfile":
        %cd /home/zharmad/host/projects/Ubq-md
    elif systemExample == "UbqAthi":
        %cd /home/zharmad/host/shared-colleague/Ubq-2017
    elif systemExample == "periplasmic":
        %cd /home/zharmad/projects/periplasmic/leucine-binding_protein
    elif systemExample == "caspase-1":
        %cd /home/zharmad/host/projects/caspase-1
    elif systemExample == "CFTR":
        %cd /home/zharmad/projects/cftr/DyNetAn
    else:
        %cd ..

In [ ]:
if not bPythonExport:
    nVecs=5
    outputDir=None
    if systemExample == "UbqCHARMM":
        # apo1 apo2 apo3
        state="apo1" ; dataDir = "./dynetan"        
        fileNameRoot = "%s_5x1000" % state
        stateB="apo2" ; dataDirB = dataDir
        fileNameRootB = "%s_5x1000" % stateB
    elif systemExample == "UbqTempProfile":
        state="Ubq" ; dataDir = "./dynetan"
        fileNameRoot = "ubq-Temp-profile"
        nameDataset = ["%iK" % (w*20+280) for w in range(5)]
    elif systemExample == "UbqAthi":
        # UbqI13V  UbqI23A  UbqI30A  UbqL43A  UbqL67A  UbqL69A  UbqV17A  UbqWT
        state="UbqL43A" ; dataDir = "./dynetan"
        fileNameRoot = "%s_5x200" % state
        stateB="UbqWT" ; dataDirB = dataDir
        fileNameRootB = "%s_5x200" % stateB
        nameConsensus ='L43A' ; nameConsensusB='WT'
    elif systemExample == "periplasmic":
        # apo holo
        state="apo" ; dataDir = "./dynetan"
        fileNameRoot = "%s_5x200" % state
        stateB="holo" ; dataDirB = dataDir
        fileNameRootB = "%s_5x200" % stateB
        nameConsensus ='holo/closed'
        nameConsensusB='apo/open'
    elif systemExample == "caspase-1":
        state="on-state" ; dataDir = "./dynetan"
        fileNameRoot = "%s_5x1000" % state
        stateB="off-state" ; dataDirB = dataDir
        fileNameRootB = "%s_5x1000" % stateB
        nameConsensus ='On state' ; nameConsensusB='Off state'
    elif systemExample == "CFTR":
        # Define mutant file IO locations. wt, P67L, E56K, R75Q, S945L, dF508, Q1291H, etc.
        state="dF508" ; temperature="310K"
        dataDir = "./results/%s/%s/" % (state, temperature)
        #Path where results will be written (you may want plots and data files in a new location)
        outputDir = "./results/%s/%s/analysis" % (state, temperature)
        fileNameRoot = "1to3"
        stateB="wt" ; dataDirB = "./results/%s/%s/" % (stateB, temperature)
        fileNameRootB = "1to6"
        
    else:
        raise KeyboardInterrupt
        
    #Path where results will be written (you may want plots and data files in a new location)
    if outputDir is None:
        outputDir = "./dynetan/analysis"
    fullPathRoot = os.path.join(dataDir, fileNameRoot)
    filePDBRef = fullPathRoot+"_reducedTraj.pdb"
    
    # outputDir is not currently used.
    outputPrefix = "./spectralDecomp"
    titleGraph  = "%s network" % state
    #if bComparison:
           
    bComparison = False
    if bComparison:
        fullPathRootB = os.path.join(dataDirB, fileNameRootB)
        filePDBRefB = fullPathRootB+"_reducedTraj.pdb"
        outputFileName = "./networkView-%s-comparison.html" % state
        
        titleGraph = "%s - %s comparison" % (state, stateB)

## Load the relevant data into DNAD objects

In [ ]:
print("= = = Loading input graph data...")
dnad = DNAdata()
# = = = loadFromFile will automatically output debug lines.
dnad.loadFromFile(fullPathRoot)
#mdU = mda.Universe( "./UbqWT/reference.pdb" )
mdU = mda.Universe( filePDBRef )
dnad.nodesAtmSel = mdU.atoms[ dnad.nodesIxArray ]

In [ ]:
if bComparison:
    print("= = = Loading reference data...")
    dnadB = DNAdata()
    dnadB.loadFromFile(fullPathRootB)
    mdUB = mda.Universe( filePDBRefB )
    dnadB.nodesAtmSel = mdUB.atoms[ dnadB.nodesIxArray ]
else:
    dnadB = dnad
    mdUB = mdU

In [ ]:
def compute_consensus_graph(listG):
    """
    The node_properties and edge properties are the values to take averages of.
    Assumes nodes are the same between all graphs in the list.
    Assumes that the union of all edges should be used, rather than the intersection of all edges.
    """
    nGraphs = len(listG)
    outG = nx.Graph()
    # = = = Create Nodes and Edges = = =
    for x, c in listG[0].nodes.data('name', default=None):
        outG.add_node(x, name=c, communityID=0)
        
    dictWeights={} ; dictCounts={}
    for w in range(nGraphs):
        for u, v, c in listG[w].edges.data('name', default=None):
            if (u,v) not in outG.edges():
                outG.add_edge(u,v,name=c)
                dictWeights[u,v]=listG[w].edges[u,v]['weight']
                dictCounts[u,v]=1
            else:
                dictWeights[u,v]+=listG[w].edges[u,v]['weight']
                dictCounts[u,v]+=1
    for k, v in dictWeights.items():
        outG.edges[k]['weight'] = v/dictCounts[k]
    return outG

In [ ]:
def build_test_graph(numNodes, style):
    G = nx.Graph()
    if style == 'coil':
        for i in range(numNodes-1):
            G.add_edge(i,i+1)        
    return G

In [ ]:
def basicTicker():
    return bokehModels.BasicTicker(min_interval=1, max_interval=1, num_minor_ticks=1)

## Get consensus graph and compute adjacency matrix alongside eigenvectors

In [ ]:
GConsensus = compute_consensus_graph(dnad.nxGraphs)
for n,d in dnad.nxGraphs[0].nodes(data='name'):
    GConsensus.nodes[n]['name']=d
listNames = [ d for x,d in GConsensus.nodes(data='name') ]

In [ ]:
dfTopology = build_structure_dataframe(dnad.nodesAtmSel, listNames)

In [ ]:
adjMat   = nx.convert_matrix.to_numpy_array(GConsensus, weight='weight', nonedge=0.0)
nNodes = GConsensus.number_of_nodes()

In [ ]:
palette = bokehPalettes.viridis(nVecs)
w, v = np.linalg.eigh(adjMat)
v = np.flip(v,axis=1)
# = = = Normalise v_lambda for net positive
for i in range(nVecs):
    if np.sum(v[:,i])<0:
        v[:,i] *= -1

## Plot eigenvector distributions

In [ ]:
dataFrame = pd.DataFrame( v[:,:nVecs], index=listNames)

In [ ]:
dfMerge = pd.merge( dfTopology, dataFrame, left_on='key', right_index=True )
dfMerge.to_csv(outputPrefix+'_eigenvectors.csv', index=False)

In [ ]:
source = bokehModels.ColumnDataSource(data=dict())
source.data['name'] = listNames
for i in range(nVecs):
    nameField='v%i'%i
    source.data[nameField] = v[:,i]
    #source.data[nameField] = np.sign(v[:,i])
    
fig = bokehPlotting.figure(width=1000, height=400, title=titleGraph,
                           x_axis_label='Node ID', y_axis_label='eigenvector',
                           x_range=listNames,
                           tools = "pan,wheel_zoom,box_zoom,box_select,tap,save,reset,help")
fig.xaxis.major_label_orientation = "vertical"
fig.toolbar.active_scroll = fig.select_one(bokehModels.WheelZoomTool)

for i in range(nVecs):
    nameField='v%i'%i
    fig.line('name', nameField, legend_label=nameField, source=source,line_width=1, line_color=palette[i])
fig.legend.click_policy="hide"    

tooltips = [("Name", "@name")]
fig.add_tools(bokehModels.HoverTool(tooltips=tooltips))

if not bPythonExport:
    output_notebook()
else:
    output_file(outputPrefix+'_eigenvectors.html')
    
show(fig)

In [ ]:
dataFrame = pd.DataFrame( extract_spectral_heirarchy(v,nVecs, bCompress=True), index=listNames)

dataFrame.rename_axis('Heirarchy', axis=1)
dataFrame.rename_axis('Node ID', axis=0)
maxColors = dataFrame.max().max()+1
palette2 = bokehPalettes.viridis(min(maxColors,256))

source = bokehModels.ColumnDataSource( pd.DataFrame(dataFrame.stack(), columns=['membership']).reset_index() )
mapper = bokehModels.LinearColorMapper(palette=palette2, low=dataFrame.min().min()-0.5, high=dataFrame.max().max()+0.5)

In [ ]:
dfMerge = pd.merge( dfTopology, dataFrame, left_on='key', right_index=True )
dfMerge.to_csv(outputPrefix+'_node_heirarchy.csv', index=False)

## Plot heirarchical divisions in sequence space

In [ ]:
fig = bokehPlotting.figure(width=1000, height=400, title=titleGraph,
                           x_axis_label='Node ID', y_axis_label='Heirarchy level',
                           x_range=list(dataFrame.index),
                           tools = "pan,wheel_zoom,box_zoom,box_select,tap,save,reset,help")
fig.grid.grid_line_color = None
fig.axis.axis_line_color = None
fig.xaxis.major_label_orientation = "vertical"
fig.yaxis.ticker = basicTicker()
fig.axis.major_tick_line_color = None
fig.toolbar.active_scroll = fig.select_one(bokehModels.WheelZoomTool)

cir = fig.rect(x="level_0", y="level_1", width=1, height=1,source=source,fill_color={'field': 'membership', 'transform': mapper},line_color=None)
cbar = bokehModels.ColorBar(title='Group Membership', color_mapper=mapper, border_line_color=None, location=(0, 0))
fig.add_layout(cbar, 'right')

tooltips = [("Name", "@level_0"),("Member","@membership")]
fig.add_tools(bokehModels.HoverTool(tooltips=tooltips))

if not bPythonExport:
    output_notebook()
else:
    output_file(outputPrefix+'_node_heirarchy.html')
    
show(fig)

In [ ]:
dataFrame = pd.DataFrame( compute_eigenvector_overlap(v,nVecs) )

dataFrame.rename_axis('eigenvector', axis=1)
dataFrame.rename_axis('eigenvector', axis=0)

dataFrame.to_csv(outputPrefix+'_overlap.csv', index=True)

palette3 = bokehPalettes.viridis(256)
source = bokehModels.ColumnDataSource( pd.DataFrame(dataFrame.stack(), columns=['overlap']).reset_index() )
mapper = bokehModels.LinearColorMapper(palette=palette3, low=dataFrame.min().min(), high=dataFrame.max().max())

In [ ]:
fig = bokehPlotting.figure(width=800, height=800, title=titleGraph,
                           x_axis_label='eigenvector', y_axis_label='eigenvector',
                           tools = "pan,wheel_zoom,box_zoom,box_select,tap,save,reset,help")
fig.grid.grid_line_color = None
fig.axis.axis_line_color = None
fig.axis.major_tick_line_color = None
fig.xaxis.ticker = basicTicker()
fig.yaxis.ticker = basicTicker()

cir = fig.rect(x="level_0", y="level_1", width=1, height=1,source=source,fill_color={'field': 'overlap', 'transform': mapper},line_color=None)

cbar = bokehModels.ColorBar(title='Overlap', color_mapper=mapper, border_line_color=None, location=(0, 0))
fig.add_layout(cbar, 'right')

tooltips = [("Pair","@level_0,@level_1"),("Overlap", "@overlap")]
fig.add_tools(bokehModels.HoverTool(tooltips=tooltips))

if not bPythonExport:
    output_notebook()
else:
    output_file(outputPrefix+'_overlap.html')

show(fig)